# Version 2: Web Scrapping from Google Maps Search: 
> This version is focused more on pulling data from google maps. 
> Again the example will be web scrapping pharmacy locations in San Antonio and dallas. 

************************
The Python libraries being used are: 
- Selenium: is an automation tool
  - Note using Selenium need to download the ChromeDriver: https://chromedriver.chromium.org/home
************************

Most of this is coming from the sources below - so if you have any questions please go to the links below: 
sources: 
1. https://www.geeksforgeeks.org/how-to-scrape-data-from-google-maps-using-python/
2. https://chromedriver.chromium.org/home
3. https://www.selenium.dev/documentation/webdriver/elements/locators/
4. https://www.selenium.dev/documentation/webdriver/elements/finders/
  



In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time 


In [5]:
# define url
url = "https://www.google.com/search?q=san+antonio+pharmacies+addresses&biw=2560&bih=1329&tbm=lcl&ei=gtYYZKWKNYvJkPIPvJKNwAw&oq=san+antonio+pharmacies+address&gs_lcp=Cg1nd3Mtd2l6LWxvY2FsEAMYADIFCCEQoAEyBQghEKABMggIIRAWEB4QHTIICCEQFhAeEB0yCAghEBYQHhAdMggIIRAWEB4QHToFCAAQgAQ6BggAEBYQHjoICAAQFhAeEAo6BQghEKsCUIgHWOcUYJEeaABwAHgAgAHaAYgBqA2SAQUwLjcuMpgBAKABAcABAQ&sclient=gws-wiz-local#rlfi=hd:;si:;mv:[[29.665743,-98.41041410000001],[29.3292076,-98.69875239999999]];tbs:lrf:!1m4!1u45!2m2!46m1!1e2!1m4!1u45!2m2!46m1!1e1!1m4!1u17!2m2!17m1!1e2!1m4!1u3!2m2!3m1!1e1!2m13!1e45!4m2!46m1!1e-1!4m2!46m1!1e0!4m2!46m1!1e1!4m2!46m1!1e2!2m1!1e3!2m4!1e17!4m2!17m1!1e2!3sIAE,lf:1,lf_ui:3"

# define column names - it is prepopulated with the column names but note that the column Address and Phone Number is going to be split into two columns later
column_list = ["Pharmacy Name", "Type of Location", "Address and Phone Number", "Hours", "Shopping_type", "Shopping_type_2", "Shopping_type_3"]

# define the file name for the exported csv file - extension .csv is added automatically through the function so only the name is required
file_name = "pharmacies_san_antonio"  


In [6]:
def google_web_scrapping(url, column_list, file_name):
    
    # define the driver -- Update the path to your chromedriver if needed
    driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")
    
    driver.get(url)

    # maximize the window
    driver.maximize_window()

    # wait period
    driver.implicitly_wait(40) # dependent on internet speed

    # move through each pages and get the data use next button where id = 'pnnext' to move to the next page
    # in the class = 'rllt__details" get data
    data_list = []
    while True:
        data_loc = driver.find_elements(By.CLASS_NAME,"rllt__details")
        for i in data_loc:
            data_list.append(i.text)
        try:
            next_page = driver.find_element(By.ID,"pnnext")
            next_page.click()
            time.sleep(5) # dependent on internet speed
        except:
            break

    data_list = [i.split('\n') for i in data_list]

    df = pd.DataFrame(data_list, columns = column_list)

    # update the dataframe by splitting the "Address and Phone Number" column into two columns if there is a "·" in the column
    df['Address'] = df.iloc[:,2].apply(lambda x: x.split('·')[0] if '·' in x else x)
    df['Phone Number'] = df.iloc[:,2].apply(lambda x: x.split('·')[1] if '·' in x else x)

    # dropping the Hours Sh column by using the index
    df.drop(df.columns[3:7], axis = 1, inplace = True)


    # export the dataframe to a csv file
    df.to_csv(file_name + '.csv', index = False)

    # return the dataframe
    return df

# call the function
df = google_web_scrapping(url, column_list, file_name)

# print the dataframe
df




C:\Users\mespinoza\AppData\Local\Temp\ipykernel_21188\3783173179.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")


,Pharmacy Name,Type of Location,Address and Phone Number,Address,Phone Number
0,H-E-B Pharmacy,Pharmacy,"San Antonio, TX · (210) 341-3875","San Antonio, TX",(210) 341-3875
1,Walgreens Pharmacy,Pharmacy,"San Antonio, TX · (210) 424-3644","San Antonio, TX",(210) 424-3644
2,Walgreens Pharmacy,Pharmacy,2425 Thousand Oaks Dr · (210) 402-2927,2425 Thousand Oaks Dr,(210) 402-2927
3,H-E-B Specialty Pharmacy Business Office,Pharmacy,(877) 432-9355,(877) 432-9355,(877) 432-9355
4,Walgreens Pharmacy,Pharmacy,6901 San Pedro Ave · (210) 349-9809,6901 San Pedro Ave,(210) 349-9809
...,...,...,...,...,...
315,MedPost Urgent Care - San Antonio West Hildebrand,Urgent care center,115 Angeles Dr Suite 103 Suite 103 · (210) 664...,115 Angeles Dr Suite 103 Suite 103,(210) 664-0962
316,San Antonio,Train station,350 Hoefgen Ave · (800) 872-7245,350 Hoefgen Ave,(800) 872-7245
317,Cibolo Creek Veterinary Hospital,Animal hospital,"Boerne, TX · (830) 816-5123","Boerne, TX",(830) 816-5123
318,Cibolo Small Animal Hospital,Veterinarian,"Cibolo, TX · (210) 651-1384","Cibolo, TX",(210) 651-1384
